## Code submission example

This competition is different than a standard Code competition. It uses a backend to serve chunks of the test data, which your model must run inference on before the next chunk is served. Because of this, you have to include some required code for your submission to work.

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in `mcts_gateway` will run in a different container with direct access to the hidden test set and hand off the data in batches of 100.

Your code will always have access to the published copies of the files.

In [ ]:
import os
import sys

import polars as pl
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from pathlib import Path

import polars as pl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# XGBoost, CatBoost, LightGBM
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb


comp_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')

### Critical import

In [ ]:
import kaggle_evaluation.mcts_inference_server

### Building a model

This example trains a model inline, but you will probably want to train a model offline, import it, and just use the notebook for inference.

It's wrapped in a function because this Code format requires you to get to the `predict` function within 15 minutes. So, if you have something expensive to do up-front (e.g., training a model, loading a model, etc.), and it will take longer than 15 minutes, those operations need to happen in the first call of `predict` so the gateway server doesn't time out.

In [ ]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))


In [ ]:
%cd /kaggle/working/

In [ ]:
%pwd

In [ ]:
target = 'utility_agent1'

# 读取数据
train = pl.read_csv(comp_path / 'train.csv')
y_train = train[target]

# 删除不需要的列
cols_to_drop = ['num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', target]
train = train.drop(cols_to_drop)

# 提取字符串类型的列进行编码
obj_cols = train.select(pl.col(pl.String)).columns
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-999, encoded_missing_value=-9999)
enc.fit(train[obj_cols])
train_transformed = enc.transform(train[obj_cols])

for e, c in enumerate(obj_cols):
    train = train.with_columns(pl.Series(c, train_transformed[:, e]))

# 划分训练集和测试集
X_train, X_test, y_train_split, y_test_split = train_test_split(train, y_train, test_size=0.2, random_state=42)

global best_model
    
# 定义模型
models = {
        "RandomForest": RandomForestRegressor(n_estimators=100, max_depth=5, n_jobs=-1),
        "ExtraTrees": ExtraTreesRegressor(n_estimators=100, max_depth=5, n_jobs=-1),
        "XGBoost": xgb.XGBRegressor(n_estimators=100, max_depth=5, n_jobs=-1),
        "LightGBM": lgb.LGBMRegressor(n_estimators=100, max_depth=5, n_jobs=-1)
}

# 存储模型的 RMSE
rmse_scores = {}

# 训练并评估每个模型
for model_name, model in models.items():
    model.fit(X_train, y_train_split)
    y_pred = model.predict(X_test)
    rmse_scores[model_name] = rmse(y_test_split, y_pred)
    print(f'{model_name} RMSE: {rmse_scores[model_name]:.4f}')

# 找到RMSE最低的模型
best_model_name = min(rmse_scores, key=rmse_scores.get)
global best_model
best_model = models[best_model_name]
print(f'Best model: {best_model_name} with RMSE: {rmse_scores[best_model_name]:.4f}')


In [ ]:


def train_and_evaluate():
    pass

### Inference should be in a function named `predict` as similar to the following:

If you're not doing anything "expensive" in the first `predict` call, you don't technically need the counter code.

In [ ]:
counter = 0
def predict(test, submission):
    global counter
    if counter == 0:
        # Perform any additional slow steps in the first call to `predict`
        train_and_evaluate()
    counter += 1    
    test_transformed = enc.transform(test[obj_cols])
    for e, c in enumerate(obj_cols):
        test = test.with_columns(pl.Series(c, test_transformed[:, e]))
    return submission.with_columns(pl.Series(target, best_model.predict(test)))

### Calling the gateway server

You must run the cell below within 15 minutes of the notebook re-run or the gateway will throw an error. If you need more than 15 minutes time to load your model, train your model, etc, you should do it during the very first `predict` call, as shown previously.

In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

Note that nothing past `inference_server.serve()` will be run when your submission is evaluated on the hidden test set.